In [9]:
import pandas as pd
import numpy as np
import json

from lib.glob_fix import glob

In [10]:
FIELD_SIZE = {'x':1.05, 'y':0.68}

In [11]:
df = pd.read_csv('data\wyscout\csv\events\England.csv')
df

,id,match_id,match_name,match_winner,home_team_id,away_team_id,absolute_sec,minute,second,period,...,yellow,second_yellow,through,fairplay,lost,neutral,won,accurate,not_accurate,goal_mouth_placement
0,177959171,2499719,Arsenal - Leicester City,1609,1609,1631,2.8,0,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1,177959172,2499719,Arsenal - Leicester City,1609,1609,1631,4.9,0,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
2,177959173,2499719,Arsenal - Leicester City,1609,1609,1631,6.5,0,6,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
3,177959174,2499719,Arsenal - Leicester City,1609,1609,1631,8.1,0,8,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
4,177959175,2499719,Arsenal - Leicester City,1609,1609,1631,10.3,0,10,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643145,251596409,2500098,West Ham United - Everton,1633,1633,1623,2796.7,46,36,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
643146,251596232,2500098,West Ham United - Everton,1633,1633,1623,2829.8,47,9,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
643147,251596410,2500098,West Ham United - Everton,1633,1633,1623,2831.2,47,11,2,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN
643148,251596234,2500098,West Ham United - Everton,1633,1633,1623,2832.4,47,12,2,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN


In [12]:
subevent_type_map = {
    'air_duel': 1,
    'ground_attacking_duel': 2,
    'ground_defending_duel': 3,
    'ground_loose_ball_duel': 4,
    'foul': 5,
    'hand_foul': 6,
    'late_card_foul': 7,
    'out_of_game_foul': 8,
    'protest': 9,
    'simulation': 10,
    'time_lost_foul': 11,
    'violent_foul': 12,
    'corner': 13,
    'free_kick': 14,
    'free_kick_cross': 15,
    'goal_kick': 16,
    'penalty': 17,
    'throw_in': 18,
    'goalkeeper_leaving_line': 19,
    'acceleration': 20,
    'clearance': 21,
    'touch': 22,
    'cross': 23,
    'hand_pass': 24,
    'head_pass': 25,
    'high_pass': 26,
    'launch': 27,
    'simple_pass': 28,
    'smart_pass': 29,
    'reflexes': 30,
    'save_attempt': 31,
    'free_kick_shot': 32,
    'shot': 33,
}

event_type_map = {
    'duel': 1,
    'foul': 2,
    'free_kick': 3,
    'goalkeeper_leaving_line': 4,
    'offside': 5,
    'others_on_the_ball': 6,
    'pass': 7,
    'interruption': 8,
    'save_attempt': 9,
    'shot': 10,
}

df['subtype_id'] = df['subtype_name'].map(subevent_type_map)
df['type_id'] = df['type_name'].map(event_type_map)

In [13]:
df.fillna(0)[['type_name', 'subtype_name', 'subtype_id']].value_counts()

type_name                subtype_name             subtype_id
pass                     simple_pass              28.0          251405
duel                     ground_attacking_duel    2.0            53859
                         ground_defending_duel    3.0            53737
                         air_duel                 1.0            37760
others_on_the_ball       touch                    22.0           34409
duel                     ground_loose_ball_duel   4.0            31332
interruption             ball_out_of_the_field    0.0            27331
pass                     high_pass                26.0           25067
                         head_pass                25.0           21332
free_kick                throw_in                 18.0           17050
pass                     cross                    23.0           12251
others_on_the_ball       clearance                21.0           11784
pass                     launch                   27.0           10247
shot            

In [14]:
# A possession starts with a pass and ends when a successful pass from the opponent is made
# or when the ball goes out of play
start_new_possession = (((df['type_name'] == 'pass') * df['accurate'] + (df['type_name'] == 'free_kick')) * df.team_id).replace(0, np.NaN).fillna(method='ffill')
start_new_possession = (start_new_possession != start_new_possession.shift(1)).cumsum()
start_new_possession = start_new_possession + ((df['type_name'] == 'interruption') | (df['type_name'] == 'foul')).shift(1).fillna(0).cumsum()
df['possession_id'] = start_new_possession
df['possession_type_name'] = (df['possession_id'].diff(1).fillna(1) * df['type_name']).replace('', np.NaN).fillna(method='ffill')
df['possession_type_id'] = df['possession_type_name'].map(event_type_map)
df['possession_team_id'] = (df['possession_id'].diff(1).fillna(1) * df['team_id']).replace(0, np.NaN).fillna(method='ffill')
df['possession_start_time'] = (df['possession_id'].diff(1).fillna(1) * df['absolute_sec']).replace(0, np.NaN).fillna(method='ffill')

In [15]:
for i in range(1, 3):
    df[f'previous_action_type_id_{i}'] = df['type_id'].shift(i)
    df[f'previous_action_is_same_team_{i}'] = (df['team_id'] == df['team_id'].shift(i)).astype(int)
    df[f'previous_action_is_same_possession_{i}'] = (df['possession_id'] == df['possession_id'].shift(i)).astype(int)
    df[f'previous_action_is_same_player_{i}'] = (df['player_id'] == df['player_id'].shift(i)).astype(int)
    df[f'previous_action_x_{i}'] = abs((100 * (1-df[f'previous_action_is_same_team_{i}'])) - df['x'].shift(i))
    df[f'previous_action_y_{i}'] = abs((100 * (1-df[f'previous_action_is_same_team_{i}'])) - df['y'].shift(i))
    df[f'previous_action_time_since_{i}'] = df['absolute_sec'] - df['absolute_sec'].shift(i)
    df[f'previous_action_x_displacement_{i}'] = df['x'] - df[f'previous_action_x_{i}']

df['possession_start_is_same_team'] = (df['possession_team_id'] == df['team_id']).astype(int)
df['possession_start_action_x'] = (df['possession_id'].diff(1).fillna(1) * df['x']).replace(0, np.NaN).fillna(method='ffill')
df['possession_start_action_y'] = (df['possession_id'].diff(1).fillna(1) * df['y']).replace(0, np.NaN).fillna(method='ffill')
df['possession_start_time_since'] = df['absolute_sec'] - df['possession_start_time']
df['possession_start_x_displacement'] = df['x'] - df['possession_start_action_x']

In [16]:
df['start_distance_to_goal'] = np.sqrt(((df['x'] - 100) * FIELD_SIZE['x'])**2 + ((df['y'] - 50) * FIELD_SIZE['y'])**2)
df['start_angle_to_goal'] = abs(np.arctan2((df['y'] - 50) * FIELD_SIZE['y'], (df['x'] - 100) * FIELD_SIZE['x']))
df['end_distance_to_goal'] = np.sqrt(((df['end_x'] - 100) * FIELD_SIZE['x'])**2 + ((df['end_y'] - 50) * FIELD_SIZE['y'])**2)
df['end_angle_to_goal'] = abs(np.arctan2((df['end_y'] - 50) * FIELD_SIZE['y'], (df['end_x'] - 100) * FIELD_SIZE['x']))

df['intent_progressive'] = ((df['type_name'] == 'pass') * (df['end_distance_to_goal'] < df['start_distance_to_goal'])).astype(int)

In [ ]:
df['shot_assist'] = (((df['type_name'].isin(['pass', 'free_kick']) & (df['accurate'] == 1)) & ((df['type_name'].shift(1) == 'shot') | (df['type_name'].shift(2) == 'shot'))).diff() < 0).shift(-1).fillna(0).astype(int)

In [ ]:
df['goal'] = df['goal'].fillna(0)

actions_before_goal = None
actions_before_own_goal = None
for i in range(10):
    if actions_before_goal is None:
        actions_before_goal = df.goal.shift(-(i))
        actions_before_own_goal = -df.own_goal.shift(-(i))
    else:
        actions_before_goal += df.goal.shift(-(i))
        actions_before_own_goal -= df.own_goal.shift(-(i))
actions_before_goal = actions_before_goal.fillna(0)
actions_before_own_goal = actions_before_own_goal.fillna(0)

is_same_period = (df.goal * df.period).replace(to_replace=False, method='bfill') == df.period
is_same_game = (df.goal * df.match_id).replace(to_replace=False, method='bfill') == df.match_id
is_team_next_goal = 2 * ((df.goal * df.team_id).replace(to_replace=False, method='bfill') == df.team_id) - 1
is_team_next_goal *= actions_before_own_goal

df['vaep_label_0'] = actions_before_goal * is_same_period * is_same_game * is_team_next_goal
df['vaep_label_0_scoring'] = df['vaep_label_0'].clip(0, 1)
df['vaep_label_0_conceding'] = abs(df['vaep_label_0'].clip(-1, 0))